# Preparador de prompts utilizando embeddings

Este notebook serve como exemplo de uso para embeddings com o objetivo de gerar prompts para LLM que sejam mais eficientes para auxílio na construção de uma ontologia relacioada a cibersegurança.

O embedding escolhido é BM25 (uma evolução de TF-IDF), mas há outros que podem ser utilizados, sejam outros _sabores_ de TF-IDF como possivelmente SPLADE, que parece apresentar resultados às vezes até melhores que BM25.

Um ponto positivo no uso de BM25 é ser um método caixa branca, o que minimiza a parte não _auditável_ (na falta de termo mais adequado) do nosso processo de automatização de geração de ontologias - já temos o uso de LLMs para auxiliar nessa geração, se pudermos encapsular esse uso de LLM podemos alterar componentes da ontologia buscados por meio de ajsutes de parâmetros mais simples.

De maneira geral, o BM25 recebe:

- Uma lista de strings representando os diferentes documentos a serem analisados - para nós, os _documentos_ em si são as frases referentes aos diferentes textos que pretendemos utilizar para compor a ontologia.

- Uma segunda lista de strings representando as chamadas _queries_, que são os termos que queremos utilizar para selecionar quais as strings mais relevantes do primeiro parâmetro.

Além disso, podemos refinar os resultados obtidos pelo BM25:

- Fornecemos um parâmetro extra (`top_n`) que indica quantas das frases de maior pontuação queremos considerar.

- Por fim, mais um parâmetro que serve para fornecer contexto se necessário: `n_vizinhos` que indica quantas frases vizinhas queremos selecionar às top n selecionadas. Esse parâmetro pode ser útil em determinados casos em que a definição de um termo buscado envolve o contexto textual ao redor da frase que contém a definição em si.

Portanto, um primeiro passo é obter um documento (padrão, regulamento etc.), que geralmente está em `.pdf`, transformá-lo em `.txt`, e quebrar esse documento nas frases que o compõem. Nesse repositório, utilizo o [Docling da IBM](https://docling-project.github.io/docling/) para a conversão. O módulo `processa_texto.py` se encarrega do tratamento do `.txt` em si, sendo chamado pelo módulo `texto_para_llm.py`.

Daí, por leitura/aprendizado/conhecimento, devem ser selecionadas palavras a serem utilizadas como as queries do BM25 para ranqueamento das frases do arquivo. O glossário que está sendo desenvolvido apresenta um papel importante aqui, pois ele servirá como consulta para quem não for especialista de cibersegurança especificamente.

Dependendo do termo ou componente da ontologia a ser definido, as demais parâmetros `top_n` e `n_vizinhos` podem ter valores diferentes para ajuste. Na minha experiência, para definições mais _diretas_, ter um `top_n` entre 2 e 5 e um `n_vizinhos` de 0 (indicando que não devemos levar nenhum vizinho em consideração) é suficiente. Componentes mais complexos podem requerer valores maiores para esses parâmetros.

No fim, o BM25 retorna as frases que foram selecionadas do texto original como as mais relevantes, concatenadas como uma única string. Um ponto importante aqui é que as frases estarão na ordem que estão no texto, não na ordem de relevância dada pelo BM25. O módulo responsável pelo processamento do BM25 em si é o `processa_bm25.py`, chamado pelo módulo `texto_para_llm.py`.

Vamos ver um exemplo de uso.

Na célula abaixo fazemos o import de bibliotecas úteis. pathlib lida com paths de arquivos independentemente de SO para não precisarmos nos preocupar com notação Windows ou Posix. Sempre considero útil ter em meus projetos, mas não é diretamente relevante ao embedding em questão.

O segundo import chama a classe definida no módulo texto_para_llm, que realiza a interface.

In [ ]:
from pathlib import Path

from BM25.modulos_bm25.texto_para_llm import PreparaTextoLLM

Daí, começamos de fato a lidar com os arquivos de entrada.

In [ ]:
# Obtendo o texto como lista de strings
# Conversão de pdf via Docling
input_pdf = Path(r"BM25/exemplos") / r"NBRISO-IEC 27035.pdf"
prepara_texto = PreparaTextoLLM()
lista_frases = prepara_texto.separarTextoArquivo(input_pdf)

# Criamos o processador de BM25
prepara_texto.preparaFrasesBM25(lista_frases)

Com a lista das frases em mãos e o processador de BM25 preparado, podemos fornecer os parâmetros adequados: as queries, o top_n e o n_vizinhos. Digamos que eu queira frases relacionadas ao conceito de _incidente_.

In [ ]:
# Lista de queries para o BM25
lista_queries = ["incident",
                "response",
                "assessment",
                "confirmation",
                "action",
                "responses",
                "actions",
                "activities",
                "management"]
# Top n - a quantidade de índices a serem considerados
top_n = 4
# Vizinhos - quantidade de frases "vizinhas" aos índices encontrados que serão consideradas
# Importante para contexto
vizinhos = 1

Com os parâmetros em mãos, chamamos o método `bm25TopFrases` do `PreparaTextoLLM`.

In [ ]:
contexto = prepara_texto.bm25TopFrases(lista_frases, lista_queries, top_n, -1, vizinhos)

Com essa string em mãos, poderemos fazer o RAG. Você pode utilizá-la como quiser para compor seu prompt. Recomendo, porém, verificar os textos recebidos antes de enviar a uma LLM. Eles estão adequados? Falta algo? Tem informação demais gerando ruído? Precisamos ajustar algum parâmetro? Essa é uma parte um pouco mais artesanal, mas de maneira geral diminuímos nosso espaço de busca de documentos de dezenas de páginas a uma ou duas no máximo, e nos tornamos um pouco mais independentes de especialistas para seguir com nossos processos (em especial quando também munidos do glossário).